# 倒立振子のシミュレーションを Julia で行いたい

## はじめに

### 倒立振子とは

台車の上に棒が立っていて，その棒が倒れないようにしつつ台車を動かすという制御問題のことを倒立振子（とうりつしんし，inverted pendulum）といいます．

制御や強化学習の界隈の Hello World としてよく親しまれている問題です．倒立振子のシミュレーションを自分で組むと勉強になるとされています．

### Julia とは

主に科学計算の世界で使われている比較的新しい言語です．Python や Matlab を使ってもいいのですが，今回は Julia を使います．

## 倒立振子の運動方程式

シミュレーションをするには，運動方程式がわかっている必要があります．

運動方程式の導出は物理学の問題であって，制御の問題ではないのでここでは詳しくは述べません．[Excelで学ぶ振動基礎/運動方程式の立て方](https://edu.katzlab.jp/lec/vib7h/files/vib7h_B.pdf)に解析力学を使って導出する方法が解説されているので，そちらを参照してください．

ここでは以下のようなパラメータを考えます．

* 定数
  * 台車の質量 $M$ [kg]
  * 振子の質量 $m$ [kg]
  * 重力加速度 $g$ [m/s^2]
  * 台車から振子の重心までの距離 $l$ [m]

* 変数
  * 台車の位置 $x$ [m]
  * 振子の傾き $θ$ [rad]
  * 外力 $f$ [N] (制御入力 $u$ に相当する)

摩擦や空気抵抗は考えないものとしています．

このとき，結論だけ書いてしまうと，運動方程式は次の通りです．
$$
(M + m) \ddot{x} + m l \ddot{θ} \cos θ - m l \dot{θ}^2 \sin θ = f \\\\
(ml \cos θ) \ddot{x} + ml^2 \ddot{θ} + mgl \sin θ = 0
$$

## 状態方程式を求める

さらにこれを状態空間モデルとして解釈するために，状態ベクトル $X$ を考えます．状態は $x, \dot{x}, θ, \dot{θ}$ の 4 つなので
$$
X = \begin{bmatrix} x \\ \dot{x} \\ θ \\ \dot{θ} \end{bmatrix}
$$
とします．

$X$ の微分を $X$ と $f$ で表現する必要があります．これの計算はそれなりに面倒なので，Julia の `Symbolics` パッケージを使用して計算します．

In [3]:
using Symbolics

# 変数を宣言する
@variables t f m l M x θ g

# 時間 t に対する微分作用素
D = Differential(t)

# 2回微分
DD = D ∘ D

# 2つの運動方程式を定義する
eq_1 = (M + m) * DD(x) + m * l * DD(θ) * cos(θ) - m * l * (D(θ))^2 * sin(θ) - f
eq_2 = m * l * cos(θ) * DD(x) + m * l^2 * DD(θ) + m * g * l * sin(θ)

display(eq_1)
display(eq_2)


-f + (M + m)*Differential(t)(Differential(t)(x)) + l*m*cos(θ)*Differential(t)(Differential(t)(θ)) - l*m*(Differential(t)(θ)^2)*sin(θ)

g*l*m*sin(θ) + (l^2)*m*Differential(t)(Differential(t)(θ)) + l*m*cos(θ)*Differential(t)(Differential(t)(x))

In [4]:
begin
    # x の２階微分を消去する
    eq_without_DDx = (m * l * cos(θ)) * eq_1 - (M + m) * eq_2 |> z->simplify(z,expand=true)
    display(eq_without_DDx)

    # θ の２階微分を表示する式を求める
    DDθ = Symbolics.solve_for(eq_without_DDx, DD(θ)) |> simplify
    display(DDθ)
end

-f*l*m*cos(θ) - M*g*l*m*sin(θ) - M*(l^2)*m*Differential(t)(Differential(t)(θ)) - g*l*(m^2)*sin(θ) - (l^2)*(m^2)*Differential(t)(Differential(t)(θ)) + (l^2)*(m^2)*(cos(θ)^2)*Differential(t)(Differential(t)(θ)) - (1//2)*(l^2)*(m^2)*sin(2θ)*(Differential(t)(θ)^2)

(f*cos(θ) + M*g*sin(θ) + g*m*sin(θ) + (1//2)*l*m*sin(2θ)*(Differential(t)(θ)^2)) / (-M*l - l*m + l*m*(cos(θ)^2))

In [5]:
begin
    # θ の２階微分を消去する
    eq_without_DDθ = l * eq_1 - cos(θ) * eq_2 |> z->simplify(z,expand=true)
    display(eq_without_DDθ)

    # x の２階微分を表示する式を求める
    DDx = Symbolics.solve_for(eq_without_DDθ, DD(x)) |> simplify
end

-f*l + M*l*Differential(t)(Differential(t)(x)) + l*m*Differential(t)(Differential(t)(x)) - (1//2)*g*l*m*sin(2θ) - l*m*(cos(θ)^2)*Differential(t)(Differential(t)(x)) - (l^2)*m*(Differential(t)(θ)^2)*sin(θ)

(f + (1//2)*g*m*sin(2θ) + l*m*(Differential(t)(θ)^2)*sin(θ)) / (M + m - m*(cos(θ)^2))

In [8]:
# Julia でシミュレーションを行うために，上記の式に基づいて関数を定義する

"""台車と振子の状態"""
struct State
    x::Float64 # 台車の位置 [m]
    θ::Float64 # 振子の角度 [rad]
    v::Float64 # 台車の速度 [m/s]
    ω::Float64 # 振子の角速度 [rad/s]
end

"""
系の状態と制御入力から，次の状態を求めるための関数．
`X' = f(X, u)` を満たす．

## Arguments
* `X::State` : 現在の状態
* `u::Float64` : 制御入力
"""
function cartpole(X :: State, u :: Float64) :: State
    x, θ, v, ω = X.x, X.θ, X.v, X.ω

    g = 9.8 # 重力加速度 [m/s^2]
    mc = 1.0 # 台車の質量 [kg]
    mp = 0.2 # 振子の質量 [kg]
    l = 0.5 # 台車から振子の重心までの長さ [m]

    x_acc = (u + 0.5 * g * mp * sin(2θ) + mp * l * ω^2 * sin(θ)) / (mc + mp * sin(θ)^2)
    θ_acc = - (u * cos(θ) + mc * g * sin(θ) + g * mp * sin(θ) + 0.5 * mp * l * ω^2 * sin(2θ) ) / (l * (mc + mp * sin(θ)^2))

    return State(
        v,
        ω,
        xcc,
        θ_acc
    )
end

cartpole